## Yolo

In [1]:
!pip install yt_dlp
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.1/800.1 kB 6.3 MB/s eta 0:00:00

In [2]:
import yt_dlp
import cv2
from ultralytics import YOLO

In [3]:
def download_youtube_video(url, output_path):
    ydl_opts = {
        'outtmpl': f'{output_path}/%(title)s.%(ext)s',
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best',
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        filename = ydl.prepare_filename(info)
    return filename

In [4]:
def process_video(input_path, output_path):
    model = YOLO('yolov8n.pt')

    cap = cv2.VideoCapture(input_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    for frame_count in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame, conf=0.7)
        annotated_frame = results[0].plot()

        out.write(annotated_frame)

        if frame_count % fps == 0:  # 매 초마다 진행 상황 출력
            print(f"Processed {frame_count // fps} seconds")

    cap.release()
    out.release()
    print("Video processing completed")


In [5]:
youtube_url = "https://youtube.com/shorts/qPZl-2VHvQo?si=LyB5pFfC1mM3-FS7"

input_video = download_youtube_video(youtube_url, "output")
output_video = "output/output_youtube.mp4"
process_video(input_video, output_video)

[youtube] Extracting URL: https://youtube.com/shorts/qPZl-2VHvQo?si=LyB5pFfC1mM3-FS7
[youtube] qPZl-2VHvQo: Downloading webpage
[youtube] qPZl-2VHvQo: Downloading ios player API JSON
[youtube] qPZl-2VHvQo: Downloading player 9ed4a7e1
[youtube] qPZl-2VHvQo: Downloading m3u8 information
[info] qPZl-2VHvQo: Downloading 1 format(s): 616+140
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 4
[download] Destination: output/＂아빠 간다..＂ 버려진게 믿기지 않는 댕댕이 깜놀 반응.f616.mp4
[download] 100% of    6.85MiB in 00:00:00 at 11.64MiB/s                
[download] Destination: output/＂아빠 간다..＂ 버려진게 믿기지 않는 댕댕이 깜놀 반응.f140.m4a
[download] 100% of  389.80KiB in 00:00:00 at 8.42MiB/s   
[Merger] Merging formats into "output/＂아빠 간다..＂ 버려진게 믿기지 않는 댕댕이 깜놀 반응.mp4"
Deleting original file output/＂아빠 간다..＂ 버려진게 믿기지 않는 댕댕이 깜놀 반응.f616.mp4 (pass -k to keep)
Deleting original file output/＂아빠 간다..＂ 버려진게 믿기지 않는 댕댕이 깜놀 반응.f140.m4a (pass -k to keep)


100%|██████████| 6.23M/6.23M [00:00<00:00, 20.3MB/s]


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


0: 640x384 4 persons, 88.1ms
Speed: 13.8ms preprocess, 88.1ms inference, 870.3ms postprocess per image at shape (1, 3, 640, 384)
Processed 0 seconds

0: 640x384 4 persons, 8.9ms
Speed: 4.0ms preprocess, 8.9ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 4 persons, 6.9ms
Speed: 3.1ms preprocess, 6.9ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 4 persons, 8.1ms
Speed: 3.4ms preprocess, 8.1ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 4 persons, 13.3ms
Speed: 3.1ms preprocess, 13.3ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 4 persons, 8.0ms
Speed: 3.1ms preprocess, 8.0ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 persons, 9.6ms
Speed: 4.3ms preprocess, 9.6ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 persons, 9.6ms
Speed: 3.1ms preprocess, 9.6ms inference, 2.0ms postprocess per image 